# Construyendo modelos con PyTorch

## Modelos básicos

### `torch.nn.Module` y `torch.nn.Parameter`


`torch.nn.Module` es la clase base para cualquier red neuronal en PyTorch. Cualquier clase que herede de `torch.nn.Module` debe implementar el método `forward`. El método `forward` es el que define cómo se calcula la salida de la red neuronal.

Cualquier objeto de tipo `torch.nn.Module` registra todos los parámetros de la red neuronal (los pesos y los sesgos). Estos parámetros son objetos de tipo `torch.nn.Parameter`, que es una subclase de `torch.Tensor`. Los parámetros se pueden acceder a través del método `parameters()` de la clase `Module`.




A continuación se muestra un ejemplo de cómo definir una red neuronal muy básica en PyTorch. Esta red consta de dos capas lineales y una función de activación ReLU entre ellas. En ella podemos ver la estructura básica de una red neuronal en PyTorch, con un método `__init__()` que define las capas y otros componentes de la red, y un método `forward()` donde se realiza la computación.

In [1]:
import torch

class TinyModel(torch.nn.Module):
    
    def __init__(self): # Definimos las capas como atributos 
        super().__init__()
        self.linear1 = torch.nn.Linear(100, 200) # Capa de entrada
        self.activation = torch.nn.ReLU() # Función de activación
        self.linear2 = torch.nn.Linear(200, 10) # Capa de salida
        self.softmax = torch.nn.Softmax() # Función de salida
    
    def forward(self, x): # Definimos el flujo de datos
        x = self.linear1(x) # Capa de entrada
        x = self.activation(x) # Función de activación
        x = self.linear2(x) # Capa de salida
        x = self.softmax(x) # Función de salida
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[ 0.0846, -0.0966, -0.0207,  ..., -0.0877,  0.0258,  0.0748],
        [-0.0168, -0.0218,  0.0965,  ...,  0.0276,  0.0269,  0.0505],
        [-0.0075, -0.0147, -0.0428,  ..., -0.0969, -0.0750,  0.0290],
        ...,
        [-0.0135,  0.0896,  0.0397,  ...,  0.0596, -0.0786,  0.0559],
        [ 0.0783, -0.0878, -0.0598,  ...,  0.0868,  0.0855, -0.0759],
        [-0.0231, -0.0505, -0.0874,  ..., -0.0177,  0.0247, -0.0623]],
       requires_grad=True)
Parameter containing:
tensor([-0.0923,  0.0695, -0.0967, -0.0348, -0.0268, -0.0773,  0.0502, -0.0698,
         0.0787,  0.0276,  0.0834, -0.0996, -0.0679,  0.0788,  0.0101,  0.0542,
         0.0844,  0.0276,  0.0794, -0.02

### Estilo funcional

La librería [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html) permite llamar a algunos de los elementos (típicamente funciones de activación) directamente como funciones en lugar de cómo atributos de un objeto. Por ejemplo, el modelo anterior es equivalente al siguiente.

In [2]:
import torch
import torch.nn.functional as F 

class TinyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(100, 200)
        self.linear2 = torch.nn.Linear(200, 10) 

    def forward(self, x):
        x = self.linear1(x) 
        x = F.relu(x)
        x = self.linear2(x)
        x = F.softmax(x)
        return x
    
    # o también es equivalente a:
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.softmax(self.linear2(x))
        return x
    
    # o aún más compacto:
    def forward(self, x):
        return F.softmax(self.linear2(F.relu(self.linear1(x))))

### Usando `Secuential`

`nn.Sequential` es una clase que permite definir una red neuronal secuencialmente. Es decir, se pueden definir las capas de la red neuronal en el orden en el que se van a aplicar. A continuación se muestra cómo se puede definir el modelo anterior usando `nn.Sequential`.

In [3]:
import torch.nn as nn

class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential( # Definimos las capas en orden como un único atributo 
        nn.Linear(100, 200),
        nn.ReLU(),
        nn.Linear(200, 10),
        nn.Softmax()
    )

  def forward(self, x):
    x = self.model(x) # Se llama a toda la secuencia, su orden ya está definido internamente
    return x


## Capas lineales

El tipo más básico de capa de red neuronal es una capa *lineal* o *totalmente conectada*. Esta es una capa en la que cada entrada influye en cada salida de la capa en un grado especificado por los pesos de la capa. Si un modelo tiene *m* entradas y *n* salidas, los pesos serán una matriz *m* x *n*.

Se llama *lineal* porque la salida de la capa es una combinación lineal de las entradas $y=Wx+b$, donde $W$ es la matriz de pesos, $x$ es el vector de entradas y $b$ es el vector de sesgos.

Si tenemos 3 entradas $x_1$, $x_2$ y $x_3$ y 2 salidas $y_1$ y $y_2$, la salida de la capa será:

$$\begin{bmatrix} y_1 \\ y_2 \end{bmatrix} = \begin{bmatrix} w_{11} & w_{12} & w_{13} \\ w_{21} & w_{22} & w_{23} \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}$$

In [4]:
lin = torch.nn.Linear(3, 2) # 3 entradas, 2 salidas

print('Parámetros de la capa (pesos y sesgos):')
for param in lin.parameters():
    print(param)

# Lo mismo accediendo directamente a cada atributo:
print('\n\nPesos:', lin.weight)
print('\n\nSesgos:', lin.bias)

x = torch.rand(1, 3) # Tensor de entrada de 1x3
print('\n\nInput:', x)

y = lin(x)
print('\n\nOutput:')
print(y)

Parámetros de la capa (pesos y sesgos):
Parameter containing:
tensor([[-0.4722,  0.5522, -0.5758],
        [-0.0792, -0.0225,  0.4340]], requires_grad=True)
Parameter containing:
tensor([-0.2049, -0.1315], requires_grad=True)


Pesos: Parameter containing:
tensor([[-0.4722,  0.5522, -0.5758],
        [-0.0792, -0.0225,  0.4340]], requires_grad=True)


Sesgos: Parameter containing:
tensor([-0.2049, -0.1315], requires_grad=True)


Input: tensor([[0.8024, 0.1478, 0.6341]])


Output:
tensor([[-0.8672,  0.0768]], grad_fn=<AddmmBackward0>)


Podemos ver que `lin.weight` contiene la matriz de pesos y que `lin.bias` contiene el vector de sesgos, siendo ambos de tipo `Parameter`.

`Parameter` es una subclase de `Tensor` que se utiliza para indicar que un tensor es un parámetro de una red neuronal y, por lo tanto, debe registrar los gradientes por el módulo de autograd de PyTorch. Esto es importante para que PyTorch pueda calcular los gradientes de los parámetros durante el entrenamiento.

## Funciones de activación

Si todo lo que hiciéramos fuera multiplicar tensores por los pesos de las capas repetidamente, solo podríamos simular funciones lineales; además, no tendría sentido tener muchas capas, ya que toda la red se reduciría a una sola multiplicación de matrices. Insertar funciones de activación no lineales entre capas es lo que permite que un modelo de aprendizaje profundo simule cualquier función, en lugar de solo las lineales.

Las funciones de activación más comunes son la función sigmoide, la tangente hiperbólica y la función **ReLU**. La función sigmoide y la tangente hiperbólica se utilizan a menudo en redes neuronales más antiguas, pero la función ReLU es la más común en la actualidad. La función ReLU es simplemente $f(x) = \max(0, x)$, lo que significa que si la entrada es negativa, la salida es cero, y si la entrada es positiva, la salida es igual a la entrada.

En clasificación binaria, la función de activación **sigmoide** es comúnmente utilizada en la capa de salida, ya que la salida de la función sigmoide está en el rango [0, 1], lo que es adecuado para representar probabilidades. En clasificación multiclase, la función de activación softmax es comúnmente utilizada en la capa de salida, ya que la salida de la función softmax es un vector de probabilidades que suman 1.

En clasificación multiclase, la función de activación ***softmax*** es comúnmente utilizada en la capa de salida, que es una generalización de la función sigmoide para múltiples clases. La función softmax asigna a cada clase una probabilidad entre 0 y 1, de forma que la suma de las probabilidades de todas las clases es 1.

## Fuentes

- https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html